# Подсчет частотности пар товаров в продуктовых чеках

В файле содержится информация о покупках людей.

* id – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)
* Товар – наименование товара
* Количество – число единиц купленного товара

Воспользуйтесь этими данными и выясните, какие пары товаров пользователи чаще всего покупают вместе. По сути, вам необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, для удобства пользователей и увеличения выручки.

# **Худший вариант по производительности. Не получилось провести необходимые вычисления на всем массиве данных за разумный отрезок времени!**

In [1]:
%%capture
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS db;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE db;'

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
user = 'postgres'
password = 'postgres'
host = 'localhost'
post = 5432
name = 'db'

In [4]:
point = 'postgresql://{}:{}@{}:{}/{}'.format(user, password, host, post, name)

In [5]:
con = create_engine(point)

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [6]:
# d = {'invoiceno': [1,1,1,1,2,2,3,3,3], 'stockcode': [2,1,3,3,4,5,3,1,5]}
# df_test = pd.DataFrame(data=d)
# df_test.to_sql("sales_test", con, if_exists='replace', index=False, method='multi')

In [38]:
df_test = pd.read_csv('/content/data_two_columns.csv')

In [39]:
df_test.shape

(541909, 2)

In [40]:
df_test = df_test.iloc[:5000]

In [41]:
df_test.to_sql("sales_test", con, if_exists='replace', index=False, method='multi')

In [42]:
def select_postgresql(sql):
    return pd.read_sql(sql, con)

In [43]:
sql = """select s.* from sales_test as s limit 10"""

In [44]:
print(select_postgresql(sql))

  invoiceno stockcode
0    536365    85123A
1    536365     71053
2    536365    84406B
3    536365    84029G
4    536365    84029E
5    536365     22752
6    536365     21730
7    536366     22633
8    536366     22632
9    536367     84879


In [45]:
sql = """with tbl_stockcode as (select distinct s.stockcode from sales_test as s),

              tbl_couple_codes as (select t1.stockcode as stockcode1, 
                                          t2.stockcode as stockcode2
                                   from tbl_stockcode as t1 cross join tbl_stockcode as t2 
                                   where t1.stockcode < t2.stockcode),

              tbl_group_codes as (select t.list_stockcode
                                  from (select s.invoiceno, array_agg(distinct s.stockcode) as list_stockcode
                                       from sales_test as s
                                       group by s.invoiceno) as t
                                  group by t.list_stockcode)

              select t1.stockcode1, t1.stockcode2, count(t2.list_stockcode)
              from tbl_couple_codes as t1 left join tbl_group_codes as t2
                                                   on t1.stockcode1 = any(t2.list_stockcode) 
                                                      and t1.stockcode2 = any(t2.list_stockcode)
              where t2.list_stockcode is not null
              group by t1.stockcode1, t1.stockcode2
              order by count(t2.list_stockcode) desc
              limit 10;"""

In [46]:
%%time
print(select_postgresql(sql))

  stockcode1 stockcode2  count
0      22865      22866     17
1     84029E     84029G     17
2     84029E     85123A     16
3     84029G     85123A     16
4      71053     85123A     14
5      22632      22865     14
6      71053     84029G     13
7      71053     84029E     13
8      37370     84029E     13
9      22086      22910     13
CPU times: user 1.12 s, sys: 136 ms, total: 1.26 s
Wall time: 3min 49s
